In [5]:
import time

def benchmark_sage(n, k, iterations):
    times = []
    for _ in range(iterations):
        start_time = time.time()
        binomial(n, k)
        end_time = time.time()
        times.append(end_time - start_time)
    
    avg_time = sum(times) / iterations
    print(f"SageMath: Tempo médio para binomial({n}, {k}) em {iterations} iterações: {avg_time:.6f} segundos")

# Parâmetros do benchmark
n = 13456
k = 345
iterations = 1000

benchmark_sage(n, k, iterations)

SageMath: Tempo médio para binomial(13456, 345) em 1000 iterações: 0.000045 segundos


In [6]:
import time
from scipy.special import comb

def benchmark_python(n, k, iterations):
    times = []
    for _ in range(iterations):
        start_time = time.time()
        comb(n, k, exact=True)
        end_time = time.time()
        times.append(end_time - start_time)
    
    avg_time = sum(times) / iterations
    print(f"Python: Tempo médio para comb({n}, {k}) em {iterations} iterações: {avg_time:.6f} segundos")

# Parâmetros do benchmark
n = 13456
k = 345
iterations = 1000

benchmark_python(n, k, iterations)

Python: Tempo médio para comb(13456, 345) em 1000 iterações: 0.000112 segundos
